# MACROTABLE

In [1]:
import pandas as pd

## INDEX

In [2]:
index = pd.read_csv('data/index.zip') #reading index data

In [3]:
#Where locality is missing subregion2 is not and viceversa --> cannot assure that these 2 are equal.
#Drop both locality and subregion as these are not adding anything to country level analysis (and we already have subregion1).
index = index.drop(columns = ['locality_code', 'locality_name']) 
index = index.drop(columns = ['subregion2_code', 'subregion2_name'])

In [4]:
#These are unique value keys, so there is no way to impute them
#They are not related at all between each other (none is derived from the other)
#Do not provide any value for country, so they can be dropped
index = index.drop(columns = ['place_id', 'wikidata_id', 'datacommons_id']) 

In [5]:
# The columns iso_3166_1_alpha_2, iso_3166_1_alpha_3 and aggregation_level are not provivding any value to our analysis
## So we drop them
index = index.drop(columns = ['iso_3166_1_alpha_2', 'iso_3166_1_alpha_3', 'aggregation_level'])

## HEALTH

In [6]:
health = pd.read_csv('data/health.zip') #1. Reading health data

In [7]:
health = health.dropna(axis = 'columns', how = 'all') #2. dropping all empty columns

## VACCINATIONS

In [8]:
vaccinations = pd.read_csv('data/vaccinations.zip') #1. Reading vaccinations data

In [9]:
vaccinations = vaccinations.dropna(axis = 'columns', how = 'all') #2. dropping all empty columns

In [10]:
vaccinations = vaccinations.fillna(0) 
# All missing values in the new_persons_fully_vaccinated column are for the first day (or the day previous to the first) of vaccinations 
## within the time perod of the dataset. 
# Therefore these must be 0, hence we impute these missing values with 0s as such.

In [11]:
vaccinations['date'] = pd.to_datetime(vaccinations['date'], format="%Y-%m-%d") #4. Changing date to datetime format

Transforming vaccinations to be able to merge it:

In [12]:
# Changing the column date from day to week:
vaccinations['date'] =  vaccinations['date'].dt.to_period("W")

In [13]:
vaccinations= vaccinations.groupby(['date','location_key'])[['new_persons_fully_vaccinated', 'cumulative_persons_fully_vaccinated']].sum()
# Grouping by date (week) and location_key, as these are the indices we want.
#We use the sum metric for the group by.

In [14]:
# Formatting the index
vaccinations = vaccinations.reset_index()

## DEMOGRAPHICS

In [15]:
demographics = pd.read_csv('data/demographics.zip') #1. Reading demographics data

In [16]:
demographics = demographics.dropna(axis = 'columns', how = 'all') #2. dropping all empty columns

In [17]:
#We also drop the population density column, as it has over 90% missing values,
## and given that this column is not really aggregatable at a country level 
### (it is not aggregatable as it is a ratio and not an absolute population value, unlike the other population columns).
demographics = demographics.drop(columns = ['population_density'])

In [18]:
# The rest of columns with null values we will clean once we have merged all the datasets. 
# This is because, these are absolute population variables that can be aggregated at a country level. 
# In other words, in doesn't matter if we don't clean them now, because once we group by country name in the macrotable (using the sum as a metric), 
## the null values will disappear (as they are not taken into account in the grouping). 

# HOSPITALIZATIONS

Through analysis of the hospitalizations dataset, we discovered there were only 7 location keys (all corresponding to locations within the United States), whereas in other tables (such as index) there were 5121, for all (4) countries.

If we merge the hospitalizations dataset, this would cause there to be many null values (for all countries except the US). This is not feasible as per the assignment instructions (that the macrotable cannot have any null/missing values). A solution to this issue would be to remove those null values in the macrotable, however, in that case we would have a macrotable with very few rows (and with values only for the US).

Even though we recognize the value of the hospitalization data:
 1. It is not essential to predict deaths (which is our final objective)
 2. We prioritize a large dataset covering all countries over having a small macrotable covering data for the US only.

 In conclusion, we decied not merge the hospitalizations table to our macrotable, and drop it from our analysis. 

## EPIDEMIOLOGY

In [19]:
epidemiology = pd.read_csv('data/epidemiology.zip') #1. Reading epidemiology data

In [20]:
epidemiology = epidemiology.drop(columns = ['new_tested', 'cumulative_tested'])
#2. Drop new_tested and cumulative_tested as both these columns are nearly completely null
# Also, there is no way to calculate/impute tested as we would need another column with the confirmed negative cases of covid (which we don't).

In [21]:
#As recovered columns are over 90% null, we drop these columns.
epidemiology = epidemiology.drop(columns = ['new_recovered', 'cumulative_recovered'])

In [22]:
# replacing nulls in new_confirmed by calculating its value from two consecutive cumulative_confirmed values 
for i in range(len(epidemiology)):
    if pd.isna(epidemiology.at[i, "new_confirmed"]) == True and epidemiology.at[i, "location_key"] == epidemiology.at[(i-1), "location_key"]:
        epidemiology.at[i, "new_confirmed"] = epidemiology.at[i, "cumulative_confirmed"] - epidemiology.at[i-1, "cumulative_confirmed"]
# Group by 'location_key' and find the indices of the rows with the minimum 'date' for each group
min_date_indices = epidemiology.groupby('location_key')['date'].idxmin()

# Subset the original DataFrame for rows with the minimum date for each location
min_date_rows = epidemiology.loc[min_date_indices].copy()

# Replace null values of 'new_confirmed' with 'cumulative_confirmed' for these rows
min_date_rows.loc[min_date_rows['new_confirmed'].isnull(), 'new_confirmed'] = min_date_rows['cumulative_confirmed']

# Update the original DataFrame with the modified rows for minimum dates
epidemiology.update(min_date_rows)

In [23]:

# Step 1: Sort the dataframe by location_key and date
epidemiology = epidemiology.sort_values(by=['location_key', 'date']).reset_index(drop=True)

# Step 2: Forward-fill 'cumulative_deceased', but leave all-null groups as NaN
epidemiology['cumulative_deceased'] = epidemiology.groupby('location_key')['cumulative_deceased'].transform(
    lambda group: group.ffill() if group.notna().any() else group
)

# Step 3: Calculate 'new_deceased', leaving all-null groups as NaN
epidemiology['new_deceased'] = epidemiology.groupby('location_key')['cumulative_deceased'].transform(
    lambda group: group.diff() if group.notna().any() else group
)

In [ ]:
import math

# Step 4: Clip negative values to ensure 'new_deceased' and 'new_confirmed' are non-negative
epidemiology['new_deceased'] = epidemiology['new_deceased'].clip(lower=0)
epidemiology['new_confirmed'] = epidemiology['new_confirmed'].clip(lower=0)

# Step 5: Extract region code (characters 4 and 5 from location_key)
epidemiology['region'] = epidemiology['location_key'].str[3:5]

# Step 6: Calculate the ratio 'x' (avg of new_deceased / avg of new_confirmed) per region and date
region_date_avg = epidemiology.groupby(['region', 'date']).apply(
    lambda group: group['new_deceased'].sum() / group['new_confirmed'].sum()
    if group['new_confirmed'].sum() > 0 else 0
).rename('x').reset_index()

# Step 7: Merge 'x' back into the original dataframe
epidemiology = epidemiology.merge(region_date_avg, on=['region', 'date'], how='left')

# Step 8: Impute missing 'new_deceased' values using 'x * new_confirmed' and round to nearest integer if decimal part>0.05 for the death ratio to be close to germany's ratio

epidemiology['new_deceased'] = epidemiology.apply(
    lambda row: (
        math.ceil(row['x'] * row['new_confirmed']) 
        if (row['x'] * row['new_confirmed']) % 1 > 0.05 
        else round(row['x'] * row['new_confirmed'])
    ) if pd.isna(row['new_deceased']) else row['new_deceased'],
    axis=1
)

# Step 9: Drop temporary columns if necessary
epidemiology.drop(columns=['region', 'x'], inplace=True)
f = epidemiology.sort_values(by=['location_key', 'date']).reset_index(drop=True)


C:\Users\Afonso Santos\AppData\Local\Temp\ipykernel_5596\32792377.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  region_date_avg = epidemiology.groupby(['region', 'date']).apply(


In [25]:
# Step 10: Recalculate cumulative_deceased using new_deceased
epidemiology['cumulative_deceased'] = epidemiology.groupby('location_key')['new_deceased'].transform(lambda group: group.cumsum())

In [26]:
#We change the datatype of 'date' from object to datetime64[ns] (i.e, the correct type)
epidemiology['date'] = pd.to_datetime(epidemiology['date'], format="%Y-%m-%d")

Transforming epidemiology to be able to merge it:

In [27]:
# Changing the column date from day to week:
epidemiology['date'] =  epidemiology['date'].dt.to_period("W") 

In [28]:
epidemiology= epidemiology.groupby(['date','location_key'])[['new_confirmed', 'new_deceased', 'cumulative_confirmed', 'cumulative_deceased']].sum()
#Grouping by date (week) and location_key, as these are the indices we want.
#We use the sum metric for the group by.

In [29]:
#Formatting the index
epidemiology = epidemiology.reset_index() 

## MERGING TABLES

In [30]:
merged1 = pd.merge(index, demographics , on=["location_key"], how="left")

In [31]:
intermediate_static = pd.merge(merged1, health, on=["location_key"], how="left")

In [32]:
grouped_intermediate_static = intermediate_static.groupby('country_code').agg({             
'population': "sum",                                 
'population_male': "sum",                         
'population_female': "sum",                       
'population_age_00_09': "sum",                   
'population_age_10_19': "sum",                    
'population_age_20_29': "sum",                     
'population_age_30_39': "sum",                     
'population_age_40_49': "sum",                     
'population_age_50_59': "sum",                     
'population_age_60_69': "sum",                     
'population_age_70_79': "sum",                     
'population_age_80_and_older': "sum",           
'life_expectancy': "mean", 
}).reset_index()

In [33]:
intermediate_dynamic =  pd.merge(epidemiology, vaccinations, on=['date', "location_key"], how="left")

In [34]:
intermediate_dynamic['country_code'] = intermediate_dynamic['location_key'].str[:2]

In [35]:
grouped_intermediate_dynamic = intermediate_dynamic.groupby(['country_code', 'date']).sum().reset_index()

In [36]:
macrotable =  pd.merge(grouped_intermediate_dynamic, grouped_intermediate_static, on= ["country_code"], how="left")

In [37]:
#We drop the following columns as these are not providing any value at the country level, and will make things more complicated when grouping
macrotable = macrotable.drop(columns = ['location_key', 'cumulative_confirmed', 'cumulative_deceased', 'cumulative_persons_fully_vaccinated'])

In [38]:
macrotable['country_code'] = macrotable['country_code'].replace({
    'DE': 'Germany',
    'US': 'United States',
    'ES': 'Spain',
    'IT': 'Italy'
})

In [39]:
macrotable = macrotable.rename(columns={'country_code': 'country_name'})

In [40]:
new_order = ['date',
            'country_name', 
            'population',
            'population_male',	
            'population_female',
            'population_age_00_09',	
            'population_age_10_19',	
             'population_age_20_29',	
            'population_age_30_39',	
             'population_age_40_49',	
             'population_age_50_59',	
             'population_age_60_69',	
            'population_age_70_79',	
            'population_age_80_and_older',	
            'life_expectancy',
             'new_confirmed',	
             'new_deceased',	
            'new_persons_fully_vaccinated']

In [41]:
macrotable = macrotable[new_order]

In [42]:
macrotable = macrotable.rename(columns={'date': 'week'})

In [43]:
group10_macrotable = macrotable.to_csv('group10_macrotable.csv', header=True, index = False)